## Import all modules

In [2]:
import yaml
import json
import sqlite3
from datetime import datetime
import pandas as pd
import sklearn
from json_flatten import flatten
from sklearn import tree
import numpy as np
from sklearn.cluster import KMeans


## Retrieve all bots from SQL database

In [3]:
conn = sqlite3.connect('goodbot_table.db')
cur = conn.cursor()

cur.execute("SELECT * FROM user_info WHERE bot_rating > 3.5")

#Check how many users in total
#cur.execute("SELECT * FROM user_info WHERE bot_rating IS NOT NULL")

user_info_all_result = cur.fetchall()
user_info_column_names = [description[0] for description in cur.description]
user_info_bot_dataframe = pd.DataFrame(user_info_all_result, \
                                   columns=user_info_column_names)
print(f"No of user info stored: {len(user_info_all_result)}\n")

conn.close()

No of user info stored: 101



## Load JSON attributes for goodbot into a dataframe

In [40]:
f = open('./GoodBot_parameters/Bot_088_132.json')
label = "#bot"

arr = json.load(f)

clean_arr = []

for elem in arr['result']:
    
    #print(elem)
    if elem['score'] > 0.7:
        clean_elem = {}

        clean_elem['user_screen_name'] = elem['user']['screen_name']
        clean_elem['lang'] = elem['user']['lang']
        clean_elem['hashtag'] = label
        clean_elem['user_id'] = elem['user']['id_str']
        clean_elem['score'] = elem['score']
        clean_elem['astroturf'] = elem['categories']['astroturf']
        clean_elem['fake_follower'] = elem['categories']['fake_follower']
        clean_elem['financial'] = elem['categories']['financial']
        clean_elem['other'] = elem['categories']['other']
        clean_elem['overall'] = elem['categories']['overall']
        clean_elem['self_declared'] = elem['categories']['self_declared']
        clean_elem['spammer'] = elem['categories']['spammer']
        clean_elem['recent_tweets_per_week'] = elem['analysis']['recentTweetsPerWeek'] #Normalize
        clean_elem['number_of_tweets'] = elem['analysis']['numberOfTweets'] #Normalize
        clean_elem['following'] = elem['analysis']['numberOfFollowees'] #Normalize
        clean_elem['followers'] = elem['analysis']['numberOfFollowers'] #Normalize
        clean_elem['numberOfLikes'] = elem['analysis']['numberOfLikes'] #Normalize
    
        clean_arr.append(clean_elem)
    

    #flattened_arr.append(flatten(elem))
    
df = pd.DataFrame(clean_arr)

df.to_csv('testdata.csv',index=False)

print(df)
    
f.close()

   user_screen_name lang hashtag              user_id  score  astroturf  \
0           AnhVin6   vi    #bot  1119193188114157569   0.80       0.06   
1   botFantasyNovel   ja    #bot  1402794820272214025   0.76       0.30   
2        kumamarket   ko    #bot            503803179   1.00       0.01   
3       shiori_no_A   ja    #bot           1625076079   0.95       0.01   
4      hnti_Quoll_H   ja    #bot           1015277407   0.95       0.05   
5        vimimivian   ja    #bot            452803371   0.79       0.05   
6       JavaGeekBot   en    #bot  1423655520196435969   0.71       0.24   
7   yuya_raidreapar   ja    #bot           1583971867   0.89       0.02   
8        YinYin2701   vi    #bot  1348512671486812160   0.87       0.07   
9           nyaande   ja    #bot            266520906   0.88       0.09   
10       AN_Ber_EMA   ja    #bot           2307162354   0.93       0.03   
11        Barbyee77   es    #bot            311213364   0.88       0.05   
12   autotechgadget   en 